# Logica Proposizionale

## Riferimenti

In questa sezione esploriamo la logica proposizionale sfruttando il framework HOL di nholz. [Nholz](https://github.com/domasin/nholz) è semplicemente un porting in F# di [HOL Zero](http://www.proof-technologies.com/holzero/) che a sua volta è un dimostratore interattivo di teoremi sviluppato da Mark Adams in OCaml nello stile LCF della famiglia HOL.

Sfruttiamo il linguaggio definito da HolZero per esplorare la logica proposizionale con la guida dell'[Handbook of Practical Logic and Automated Reasoning](https://www.cl.cam.ac.uk/~jrh13/atp/) di John Harrison, seguendo passo passo il capitolo sulla logica proposizionale e riadattando le funzioni lì definite al linguaggio HOL del nostro framework. 

L'Handbook di John Harrison è accompagnato da codice sorgente in OCaml che è stato portato in F# da Eric Taucher, Jack Pappas, Anh-Dung Phan ed è disponibile su Github: [fsharp-logic-examples](https://github.com/jack-pappas/fsharp-logic-examples/). Nel ridefinire le funzioni dell'handbook riadattandole alla logica HOL si è utilizzata come riferimento proprio l'implementazione in F# del codice descritto nell'Handbook.

## Introduzione

La logica proposizionale studia espressioni che intendono rappresentare proposizioni, cioè  affermazioni che possono essere considerate vere o false e che chiameremo nel seguito semplicemente "formule". All'interno del framework HOL che utilizziamo, queste sono semplicemente termini di tipo `bool` che possono essere costruite da atomi booleani, costituiti dalle costanti `true` e `false` e da variabili di tipo `bool`, a cui sono applicati i connettivi logici proposizionali `~`, `/\`, `\/`, `<=>` e `<=>`. Le proposizioni atomiche sono come le variabili nell'algebra ordinaria, e a volte ci riferiamo ad esse come variabili proposizionali o variabili booleane. Come suggerisce la parola "atomiche", non ne viene analizzata la struttura interna; questo porterebbe a considerare una logica predicativa che al momento non viene trattata. I connettivi proposizionali all'interno della logica HOL sono semplicemente funzioni da valori di verità a valori di verità.

## Avvio del motore logico

Per iniziare referenziamo il motore di nholz:

In [9]:
#I "../../src/bin/Debug/net7.0/"
#r "nholz.dll"

open HOL


e istruiamo l'interprete .NET Interactive a restituire una rappresentazione concreta della sintassi dei tipi e dei termini piuttosto che la loro sintassi astratta interna al sistema:

In [10]:
module HolTypeAndTermsFormatter =
    Formatter.SetPreferredMimeTypesFor(typeof<hol_type> ,"text/plain")

    Formatter.Register<hol_type>((fun ht -> sprintf "%A" (print_type ht)), "text/plain")

    Formatter.SetPreferredMimeTypesFor(typeof<term> ,"text/plain")

    Formatter.Register<term>((fun tm -> sprintf "%A" (print_term tm)), "text/plain")

in uno script F# inseriremmo invece:

    fsi.AddPrinter print_type
    fsi.AddPrinter print_term

Infine carichiamo almeno le teorie fino a `Bool` che contiene la definizione dei tipi e dei termini booleani e dei loro connettivi:

In [11]:
CoreThry.load
Equal.load
Bool.load

[ (eta_ax, []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("f", Tycomp ("->", [Tyvar "a"; Tyvar "b"])), Tmcomb (Tmcomb (Tmconst ("=", Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("bool", [])])])), Tmabs (Tmvar ("x", Tyvar "a"), Tmcomb (Tmvar ("f", Tycomp ("->", [Tyvar "a"; Tyvar "b"])), Tmvar ("x", Tyvar "a")))), Tmvar ("f", Tycomp ("->", [Tyvar "a"; Tyvar "b"])))))), (imp_antisym_ax, []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("p1", Tycomp ("bool", [])), Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("p2", Tycomp ("bool", [])), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmvar ("p1", Tycomp ("bool", []))), Tmvar ("p2", Tycomp ("bool", [])))), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmvar ("p2", Tycomp ("bool", []))), Tmvar ("p1", Tycomp ("bool", [])))), Tmcomb (Tmcomb (Tmconst ("=", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmvar ("p1", Tycomp ("bool", []))), Tmvar ("p2", Tycomp ("bool", [])))))))))), (select_ax, []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])]); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("P", Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])])), Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("x", Tyvar "a"), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmcomb (Tmvar ("P", Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])])), Tmvar ("x", Tyvar "a"))), Tmcomb (Tmvar ("P", Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])])), Tmcomb (Tmconst ("@", Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])]); Tyvar "a"])), Tmvar ("P", Tycomp ("->", [Tyvar "a"; Tycomp ("bool", [])])))))))))) ] HeadOrDefault (eta_ax, []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("f", Tycomp ("->", [Tyvar "a"; Tyvar "b"])), Tmcomb (Tmcomb (Tmconst ("=", Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycom... Item1 eta_ax Item2 []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("f", Tycomp ("->", [Tyvar "a"; Tyvar "b"])), Tmcomb (Tmcomb (Tmconst ("=", Tycomp ("->", [Tycomp ("->", [Tyvar "a"; Tyvar "b"]); Tycomp ("->", ... TailOrNull [ (imp_antisym_ax, []|- Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("p1", Tycomp ("bool", [])), Tmcomb (Tmconst ("!", Tycomp ("->", [Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])]); Tycomp ("bool", [])])), Tmabs (Tmvar ("p2", Tycomp ("bool", [])), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp ("bool", []); Tycomp ("->", [Tycomp ("bool", []); Tycomp ("bool", [])])])), Tmvar ("p1", Tycomp ("bool", []))), Tmvar ("p2", Tycomp ("bool", [])))), Tmcomb (Tmcomb (Tmconst ("==>", Tycomp ("->", [Tycomp (

## Operazioni sintattiche

Il modulo [`Bool`](https://domasin.github.io/nholz/reference/hol-bool.html) contiene già alcune operazioni sintattiche su formule booleane che le dividono nei loro elementi e che saranno utilizzate nel seguito.

Una prima cosa importante è poter distinguere tra espressioni atomiche ed espressioni composte. A questo scopo definiamo `is_bool_atom` come una funzione che restituisce vero per termini booleani costanti o variabili.

In [12]:
/// the term is a boolean atom
let is_bool_atom tm = 
    tm |> is_bool_term && (tm |> is_const || tm |> is_var)

In [13]:
@"(p:bool) /\ q" 
|> parse_term
|> is_bool_atom

False

In [14]:
@"(p:bool)" 
|> parse_term
|> is_bool_atom

True

Sulle formule composte vogliamo poter applicare delle funzioni sui loro atomi. A questo scopo definiamo `overatoms` per ricorsione su termini di questo genere come un analogo dell'iteratore di liste che itera una funzione binaria su tutti gli atomi di una formula.

In [15]:
let rec overatoms f tm b =
    if tm |> is_bool_atom then 
        f tm b
    elif tm |> is_not then
        let p = tm |> dest_not
        overatoms f p b
    elif tm |> is_conj then
        let (p,q) = tm |> dest_conj
        overatoms f p (overatoms f q b)
    elif tm |> is_disj then
        let (p,q) = tm |> dest_disj
        overatoms f p (overatoms f q b)
    elif tm |> is_imp then
        let (p,q) = tm |> dest_imp
        overatoms f p (overatoms f q b)
    elif tm |> is_eq then
        let (p,q) = tm |> dest_eq
        overatoms f p (overatoms f q b)
    else failwith "check type annotation on eq"

Un'applicazione particolarmente comune è quella di raccogliere qualche insieme di attributi associati agli atomi; ritornando solamente, nel caso più semplice, l'insieme di tutti gli atomi. Possiamo far questo iterando una funzione f insieme con un "append" su tutti gli atomi, e convertendo infine il risultato in un insieme per rimuovere i duplicati. 

In [16]:
let atom_union f tm =
    (tm, [])
    ||> overatoms (fun h (t) -> (f h) @ t)
    |> List.distinct |> List.sort

## Semantica della logica proposizionale

Dal momento che le formule proposizionali intendono rappresentare asserzioni che possono essere vere o false, in ultima analisi il significato di una formula è semplicemente uno dei due valori di verità "vero" e "falso". Comunque, esattamente come un'espressione algebrica x + y + 1 ha un significato definito solo quando sappiamo per che cosa stanno le variabili x e y, il significato di una formula proposizionale dipende dai valori di verità assegnati alle sue formule atomiche. Questa assegnazione è codificata in una valutazione, che è una funzione dagli insiemi degli atomi all'insieme dei valori di verità {falso,vero}. Data una formula `p` e una valutazione `v` valutiamo il valore di verità complessivo con la seguente funzione definita ricorsivamente:

In [17]:
let rec eval tm v =
    if tm = false_tm then 
        false
    elif tm = true_tm then
        true
    elif tm |> is_bool_atom then 
        v tm
    elif tm |> is_not then 
        let p = tm |> dest_not
        not <| eval p v
    elif tm |> is_conj then 
        let (p,q) = tm |> dest_conj
        (eval p v) && (eval q v)
    elif tm |> is_disj then 
        let (p,q) = tm |> dest_disj
        (eval p v) || (eval q v)
    elif tm |> is_imp then 
        let (p,q) = tm |> dest_imp
        not(eval p v) || (eval q v)
    elif tm |> is_eq then 
        let (p,q) = tm |> dest_eq
        (eval p v) = (eval q v)
    else
        failwith "Not part of propositional logic."